# 🔄 Cyclic Training/Testing for Federated Learning with CUAV Attack

## Complete Implementation with 23 Graphs

This notebook implements a **cyclic training → testing → training → testing** workflow for comprehensive evaluation of Hierarchical Federated Learning under CUAV attack scenarios.

### Architecture Overview

**Total Graphs: 23**

#### Phase 1: Single Cluster Baseline (100 rounds)
- 600 clients with Dirichlet partition (α=0.5)
- Cyclic: Train Round 1 → Test → Train Round 2 → Test → ... → Round 100
- **Output:** 2 graphs (1 training, 1 testing)

#### Phase 2: Multi-Cluster Normal Operation (100 rounds)
- 3 clusters with equal client distribution
- Training: Equal split only
- Testing: Both Equal & Dirichlet splits
- **Output:** 9 graphs
  - 1 overall training graph
  - 4 testing graphs (1 overall + 3 per-cluster) for Equal split
  - 4 testing graphs (1 overall + 3 per-cluster) for Dirichlet split

#### Phase 3: CH Compromise - Convergence Case (rounds 101-125)
- Continue from Phase 2 (after 100 rounds)
- CH0 compromised at round 111
- Testing only (no new training graphs)
- Flat lines for C0 during D&R-E phase
- **Output:** 6 graphs
  - 3 per-cluster testing graphs (C0, C1, C2) for Equal split
  - 3 per-cluster testing graphs (C0, C1, C2) for Dirichlet split

#### Phase 4: CH Compromise - Transient Case (fresh 30 rounds)
- **New training session** (restart from scratch)
- CH0 compromised at round 11
- Testing only (no new training graphs)
- **Output:** 6 graphs (same structure as Phase 3)

## 1. Import Libraries and Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pickle
import os
import json
from collections import defaultdict
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✓ Libraries imported successfully")

## 2. Configuration and Parameters

In [ ]:
# ==================== GLOBAL CONFIGURATION ====================

# Data paths
DATA_PATH = './datasets/local_cache/dataset_12500_samples_65_features.csv'
RESULTS_DIR = './results_cyclic'
os.makedirs(RESULTS_DIR, exist_ok=True)

# Model parameters
INPUT_DIM = 64  # After dropping 'label'
HIDDEN_DIM = 32
OUTPUT_DIM = 2  # Binary classification
LEARNING_RATE = 0.01

# Federated Learning parameters
NUM_CLIENTS_SINGLE = 600  # Single cluster
NUM_CLUSTERS = 3  # Multi-cluster
CLIENTS_PER_CLUSTER = 200
TOTAL_CLIENTS_MULTI = NUM_CLUSTERS * CLIENTS_PER_CLUSTER

# Data split parameters
DIRICHLET_ALPHA = 0.5  # For non-IID split
TEST_SIZE = 0.2  # 20% for testing

# Training parameters
SINGLE_CLUSTER_ROUNDS = 100
MULTI_CLUSTER_ROUNDS = 100
COMPROMISE_CONVERGENCE_ROUNDS = 25  # Rounds 101-125
COMPROMISE_TRANSIENT_ROUNDS = 30  # Fresh 30 rounds

# Attack parameters
CONVERGENCE_COMPROMISE_ROUND = 111  # During rounds 101-125
CONVERGENCE_DETECTION_ROUND = 112
CONVERGENCE_RECOVERY_START = 119
CONVERGENCE_RECOVERY_END = 121

TRANSIENT_COMPROMISE_ROUND = 11  # During fresh 30 rounds
TRANSIENT_DETECTION_ROUND = 12
TRANSIENT_RECOVERY_START = 19
TRANSIENT_RECOVERY_END = 21

# Target cluster for compromise
COMPROMISED_CLUSTER = 0  # Cluster 0 (CH0)

# Testing parameters
TEST_SAMPLES = 2500  # Per test round

print("=" * 60)
print("CONFIGURATION LOADED")
print("=" * 60)
print(f"Single Cluster: {NUM_CLIENTS_SINGLE} clients, {SINGLE_CLUSTER_ROUNDS} rounds")
print(f"Multi-Cluster: {NUM_CLUSTERS} clusters × {CLIENTS_PER_CLUSTER} clients = {TOTAL_CLIENTS_MULTI} total")
print(f"Multi-Cluster Rounds: {MULTI_CLUSTER_ROUNDS}")
print(f"Compromise Convergence: Rounds 101-125 (compromise at {CONVERGENCE_COMPROMISE_ROUND})")
print(f"Compromise Transient: {COMPROMISE_TRANSIENT_ROUNDS} rounds (compromise at {TRANSIENT_COMPROMISE_ROUND})")
print(f"Test samples per round: {TEST_SAMPLES}")
print(f"Dirichlet α: {DIRICHLET_ALPHA}")
print("=" * 60)

## 3. Data Loading and Preprocessing